As you must already be knowing that the challange with text based data is that it comprises of words and sentences.. not numeric variables :)<br>
Machine Learning, Deep Learning, and Statistical models mostly expects, most of the input data, in numerical format. That's why we need to perform feature engineering, which creates numeric features, for text based data, without loosing much infromation.

Previously, I explained how to create text based features using BOW, TF-IDF, and N-Gram techniques. But the problem with those technique is that they are very convential, which means they are pretty straight forward, and have been put together without much of the time and space optimization. In simple words, their we create vectors for word occurances or next word after each N-Grams, which eventually results in really huge vectors for a large amount of data. This vector is one hot encoded and rarely contains 1, means it carries a large amount of unnecessary information.

Here, we are going to look into a more sophisticated feature engineering technique for text based data, called Word embedding. Word embedding is basically used as one of the data preparation step while solving many of the NLP problems. Let's move forward and understand what it is and how it works:

In [1]:
!pip install tweepy

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
from numpy import array, asarray, zeros
import pandas as pd
import numpy as np

from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Flatten
from keras.layers.embeddings import Embedding

# To consume Twitter's API
import tweepy
from tweepy import OAuthHandler 

from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from textblob.np_extractors import ConllExtractor

from nltk.tokenize import word_tokenize
import re

So, what is Word Embedding?<br>
**It is most popular way of representing document vocabulary. The basic purpose of word embeddings is to capture and store the context of words with respect to document. It also stores semantic and syntactic relation with other words in a document. In computational perspective it is basically a vector which stores all the contextual, semantics and syntatic relations of that word.**

Two most commonly used technique to create a Word Embedding are:
* **GloVe**
* **Word2Vec**

There are many ways to implement these techniques, in this Kernel we'll look into the implementation using Keras

## Implementation using Keras

Keras supports two types of APIs for implementation: 
- **Sequential Methods**
- **Functional Methods**

Let's look at them one by one


### Keras Sequential Methods

Embedding() method can be used to implement a class, needed to develop an embedding layer. Keras can either be used to **create a new word embedding by learning custom words** and use them in problems like classification, sentiment analysis, etc. **or it can be used to load the pre-trained word embedding** and use them to solve our purpose.<br><br>
In this Kernel, we'll perform a simple text classification exercise by consuming the newly trained word-embeddings.


Let's jump to the implementation now,

#### Custom Implementation:

In [3]:
# keys and tokens from the Twitter Dev Console
consumer_key = 'Sec3MvclRIx2RVlgu9l0SJX6D'
consumer_secret = 'ayoPNWtBm7fWpMBoK6EwRmegu3SW8Rw9mzJkottkv97quPe941'
access_token = '736550752760406018-so5CPJrEbJKb3c3Pq8va3VFr0yk4S0E'
access_token_secret = 'Cgr8tz0h6FTU7kxAjDzpHnjffNTHxWsBytXnu4Ihd1TFb'

To perform custom word embedding and sentiment analysis, we need a decent amount of text data. We live in an era of Social Networking, nothing can be a better source of same than twitter. Let's get tweets on desired topic.

In [4]:
topic = 'pfizer'

In [5]:
# Class to incrementally pull tweets using tweepy API calls
class TwitterClient(object): 
    def __init__(self): 
        #Initialization method. 
        try: 
            # create OAuthHandler object 
            auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            # add hyper parameter 'proxy' if executing from behind proxy "proxy='http://172.22.218.218:8085'"
            self.api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
            
        except tweepy.TweepError as e:
            print(f"Error: Tweeter Authentication Failed - \n{str(e)}")

    def get_tweets(self, query, maxTweets = 1000):
        #Function to fetch tweets. 
        # empty list to store parsed tweets 
        tweets = [] 
        sinceId = None
        max_id = -1
        tweetCount = 0
        tweetsPerQry = 100

        while tweetCount < maxTweets:
            try:
                if (max_id <= 0):
                    if (not sinceId):
                        new_tweets = self.api.search(q=query, count=tweetsPerQry)
                    else:
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                since_id=sinceId)
                else:
                    if (not sinceId):
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                max_id=str(max_id - 1))
                    else:
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                max_id=str(max_id - 1),
                                                since_id=sinceId)
                if not new_tweets:
                    print("No more tweets found")
                    break

                for tweet in new_tweets:
                    parsed_tweet = {} 
                    parsed_tweet['tweets'] = tweet.text 

                    # appending parsed tweet to tweets list 
                    if tweet.retweet_count > 0: 
                        # if tweet has retweets, ensure that it is appended only once 
                        if parsed_tweet not in tweets: 
                            tweets.append(parsed_tweet) 
                    else: 
                        tweets.append(parsed_tweet) 
                        
                tweetCount += len(new_tweets)
                print("Downloaded {0} tweets".format(tweetCount))
                max_id = new_tweets[-1].id

            except tweepy.TweepError as e:
                # Just exit if any error
                print("Tweepy error : " + str(e))
                break
        
        return pd.DataFrame(tweets)

In [6]:
twitter_client = TwitterClient()

# calling function to get tweets
tweets_df = twitter_client.get_tweets(topic, maxTweets=10000)
print(f'tweets_df Shape - {tweets_df.shape}')
tweets_df.head(10)

Downloaded 100 tweets
Downloaded 200 tweets
Downloaded 300 tweets
Downloaded 400 tweets
Downloaded 499 tweets
Downloaded 599 tweets
Downloaded 699 tweets
Downloaded 799 tweets
Downloaded 899 tweets
Downloaded 999 tweets
Downloaded 1099 tweets
Downloaded 1199 tweets
Downloaded 1298 tweets
Downloaded 1398 tweets
Downloaded 1498 tweets
Downloaded 1598 tweets
Downloaded 1698 tweets
Downloaded 1798 tweets
Downloaded 1898 tweets
Downloaded 1998 tweets
Downloaded 2098 tweets
Downloaded 2198 tweets
Downloaded 2298 tweets
Downloaded 2398 tweets
Downloaded 2498 tweets
Downloaded 2598 tweets
Downloaded 2698 tweets
Downloaded 2798 tweets
Downloaded 2898 tweets
Downloaded 2998 tweets
Downloaded 3098 tweets
Downloaded 3195 tweets
Downloaded 3295 tweets
Downloaded 3395 tweets
Downloaded 3495 tweets
Downloaded 3595 tweets
Downloaded 3695 tweets
Downloaded 3795 tweets
Downloaded 3895 tweets
Downloaded 3995 tweets
Downloaded 4095 tweets
Downloaded 4195 tweets
Downloaded 4295 tweets
Downloaded 4395 tweet

,tweets
0,Mengele informuje....... A ja informuję-w DUPE...
1,RT @DrPacoMoreno1: México participara en los e...
2,RT @SStolkiner: Con Pfizer. Con Sputni...
3,RT @ilgiornale: Un'infermiera è stata ricovera...
4,RT @MichaelYeadon3: https://t.co/20AiIyJ6L9\nB...
5,RT @fatimahfnz: A 'very healthy' middle-aged d...
6,RT @mswami001: #CovidVaccine #secondshot done....
7,RT @RassNational53: Louis Fouché (anesthésiste...
8,RT @IsraeliPM: Prime Minister Netanyahu and He...
9,"RT @MonzonAgustinEz: Seba, tu tweet pegaría mu..."


We only have got tweets, let's get sentiment labels for same using TextBlob. Note that, we can use more sophisticated and much better technique for sentiment labeling, but here that's not our focus, so going with one of the simplest technique.

In [7]:
def fetch_sentiment_using_textblob(text):
    analysis = TextBlob(text)
    return 1 if analysis.sentiment.polarity >= 0 else 0

tweets_df['sentiments'] = tweets_df.tweets.apply(lambda tweet: fetch_sentiment_using_textblob(tweet))
tweets_df['sentiments'].value_counts()

1    4231
0     260
Name: sentiments, dtype: int64

Below code block peforms a cleansing of tweets we just pulled:

In [8]:
# Cleansing tweets

def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)

    return text 

# We are keeping cleaned tweets in a new column called 'tidy_tweets'
tweets_df['tidy_tweets'] = np.vectorize(remove_pattern)(tweets_df['tweets'], "@[\w]*: | *RT*")


# Removing links (http | https)
cleaned_tweets = []

for index, row in tweets_df.iterrows():
    # Here we are filtering out all the words that contains link
    words_without_links = [word for word in row.tidy_tweets.split() if 'http' not in word]
    cleaned_tweets.append(' '.join(words_without_links))

tweets_df['tidy_tweets'] = cleaned_tweets

# Removing tweets with empty text
tweets_df = tweets_df[tweets_df['tidy_tweets']!='']

# Dropping duplicate rows
tweets_df.drop_duplicates(subset=['tidy_tweets'], keep=False)

# Resetting index
tweets_df = tweets_df.reset_index(drop=True)

# Removing Punctuations, Numbers and Special characters
tweets_df['absolute_tidy_tweets'] = tweets_df['tidy_tweets'].str.replace("[^a-zA-Z# ]", "")

In [9]:
tweets_df.sample(10)

,tweets,sentiments,tidy_tweets,absolute_tidy_tweets
2543,@TVPCyber_Fraud @ThamesVP A side-by-side compa...,1,@TVPCyber_Fraud @ThamesVP A side-by-side compa...,TVPCyberFraud ThamesVP A sidebyside comparison...
2780,Voor het geval je mag kiezen welk vaccin je we...,1,Voor het geval je mag kiezen welk vaccin je we...,Voor het geval je mag kiezen welk vaccin je we...
4098,RT @TomeuRamon: Filomena ayudando a conservar ...,1,Filomena ayudando a conservar la temperatura d...,Filomena ayudando a conservar la temperatura d...
310,RT @h_helvah: A doc. who was a game fisherman ...,1,A doc. who was a game fisherman &amp; a certif...,A doc who was a game fisherman amp a certified...
3448,@DrNarram @nixcii @doctor_oxford Talk me throu...,1,@DrNarram @nixcii @doctor_oxford Talk me throu...,DrNarram nixcii doctoroxford Talk me through t...
1104,@ojosycachetes Pfizer pide condiciones inacept...,1,@ojosycachetes Pfizer pide condiciones inacept...,ojosycachetes Pfizer pide condiciones inacepta...
1379,"@blah_baa For the Pfizer vaccine, 40% of parti...",1,"@blah_baa For the Pfizer vaccine, 40% of parti...",blahbaa For the Pfizer vaccine of participant...
1009,RT @ellinikahoaxes: Κυκλοφόρησε στο Facebook ο...,1,Κυκλοφόρησε στο Facebook ο ισχυρισμός πως ο επ...,Facebook Pfizer COVID
828,"@bbimbi LAS VACUNAS SON PARA LA GILADA. LA ""MA...",1,"@bbimbi LAS VACUNAS SON PAA LA GILADA. LA ""MAC...",bbimbi LAS VACUNAS SON PAA LA GILADA LA MACOPO...
2216,"Pfizer, o Toxicroak, critando o Brick Break no...",1,"Pfizer, o Toxicroak, critando o Brick Break no...",Pfizer o Toxicroak critando o Brick Break no S...


Constructing corpus and sentiments lists for ease in further operations

In [10]:
# get a list of sentences/tweets
corpus = list(tweets_df.absolute_tidy_tweets.values)

# sentiments list
sentiments = list(tweets_df.sentiments.values)

In [11]:
# getting list of tokens
all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

In [12]:
unique_words = set(all_words)
print(len(unique_words))

16332


In [13]:
# identifying vocal lenght of unique words, it basically helps in deciding on hishing or word reference setup
vocab_length = int(len(unique_words)) + 5 # adding 5 to take some buffer
vocab_length

16337

**one_hot** method is imported from Keras, and is being used to label encode our sentences.. please note that, it's different from the conventional one hot encoding

In [14]:
embedded_sentences = [one_hot(sent, vocab_length) for sent in corpus]
print(embedded_sentences)

[[12288, 41, 11587, 6773, 12143, 12039, 3030, 14410, 10159, 14639, 9129, 6002, 9909, 1393, 8036, 13880, 6325], [124, 5112, 9768, 13654, 8252, 6464, 4577, 6421, 10340, 4715, 3614, 10340, 10155, 13872, 11587, 11949, 3985, 5113, 11587, 12420, 9571], [15932, 12420, 15932, 7461, 14216], [14576, 7035, 13924, 11587, 15846, 8171, 12226, 7535, 5280, 15692, 12420, 15932, 13423, 16008, 14675, 5932, 3262, 1703, 1616, 15362], [12806, 12205, 7014, 8531, 4249, 7444, 7807, 12780, 11144, 9393, 381, 11170, 6270, 11675, 381], [11587, 15756, 3720, 910, 974, 12192, 13832, 10116, 6290, 10984, 11660, 6216, 7762, 14344, 11234, 3091, 7177, 673, 9958, 12229], [8625, 15860, 16257, 4912, 1111, 7393, 13807, 11182, 5554, 7267, 13191, 11137, 4580, 14743, 13980], [15444, 11988, 3611, 8733, 8659, 9967, 1645, 16255, 8762, 2097, 14102, 10918, 14525, 12410, 14220, 7456], [3680, 6139, 7676, 11376, 13893, 6139, 13250, 3463, 11587, 2933, 673, 12420, 4512, 8511, 3418, 12205, 7014], [5846, 3904, 13282, 13207, 9762, 9366, 1619

In [15]:
# finding the largest sentence length
word_count = lambda sentence: len(word_tokenize(sentence))
print(word_count)
longest_sentence = max(corpus, key=word_count)
print(longest_sentence)
length_long_sentence = len(word_tokenize(longest_sentence))
print(length_long_sentence)

<function <lambda> at 0x7f6e36b63170>
VicoSotto sir im from one oasis pasig the  more vaccine we should order in pasig is russia sputnik v is also ok and rd is pfizer ng usa
28


In [16]:
# pad_sequences is being used here to make sure that all sentences are of equal length
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[12288    41 11587 ...     0     0     0]
 [  124  5112  9768 ...     0     0     0]
 [15932 12420 15932 ...     0     0     0]
 ...
 [ 7812 12473  3914 ...     0     0     0]
 [13607 16255 16305 ...     0     0     0]
 [ 1246  1092 11587 ...     0     0     0]]


Model building: We are simply adding all the basic layers i.e. Embedding, Flattening, and final output

In [17]:
model = Sequential()
model.add(Embedding(vocab_length, 100, input_length=length_long_sentence))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [18]:
# You can try with other optimizers
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 28, 100)           1633700   
_________________________________________________________________
flatten (Flatten)            (None, 2800)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2801      
Total params: 1,636,501
Trainable params: 1,636,501
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
# fitting over padded sentences
model.fit(np.asarray(padded_sentences), np.asarray(sentiments), epochs=100, verbose=1)

Epoch 1/100
139/139 [==============================] - 2s 17ms/step - loss: 0.2584 - acc: 0.9377
Epoch 2/100
139/139 [==============================] - 2s 17ms/step - loss: 0.1446 - acc: 0.9452
Epoch 3/100
139/139 [==============================] - 2s 17ms/step - loss: 0.0691 - acc: 0.9779
Epoch 4/100
139/139 [==============================] - 2s 17ms/step - loss: 0.0251 - acc: 0.9953
Epoch 5/100
139/139 [==============================] - 2s 15ms/step - loss: 0.0115 - acc: 0.9986
Epoch 6/100
139/139 [==============================] - 2s 16ms/step - loss: 0.0069 - acc: 0.9993
Epoch 7/100
139/139 [==============================] - 2s 17ms/step - loss: 0.0056 - acc: 0.9989
Epoch 8/100
139/139 [==============================] - 2s 18ms/step - loss: 0.0037 - acc: 0.9993
Epoch 9/100
139/139 [==============================] - 2s 17ms/step - loss: 0.0033 - acc: 0.9989
Epoch 10/100
139/139 [==============================] - 2s 15ms/step - loss: 0.0029 - acc: 0.9989
Epoch 11/100
139/139 [=======

In [20]:
# Evaluating over train set itself
loss, accuracy = model.evaluate(padded_sentences, np.asarray(sentiments), verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 99.977434


This is how we can build a custom word embedding matrix. Next we'll see how to consume pre-trained GloVe word embedding matrix.

### Loading Pretrained Word Embeddings

**corpus** and **sentiment** list is already constructed for our tweets. <br>
Let's look into other ways of creating tokens and padded sentences using Keras

In [21]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(corpus)

In [22]:
vocab_length = len(word_tokenizer.word_index) + 5
vocab_length

14721

In [23]:
embedded_sentences = word_tokenizer.texts_to_sequences(corpus)
embedded_sentences[:3]

[[3295,
  5795,
  4,
  802,
  5796,
  5797,
  21,
  5798,
  117,
  5799,
  3296,
  5800,
  2335,
  5801,
  5802,
  407,
  1470],
 [516,
  5803,
  13,
  49,
  1045,
  803,
  256,
  2,
  5,
  14,
  5804,
  5,
  1046,
  40,
  4,
  1471,
  3297,
  458,
  4,
  1,
  113],
 [31, 1, 31, 242, 195]]

In [24]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

print(padded_sentences[:3])

[[3295 5795    4  802 5796 5797   21 5798  117 5799 3296 5800 2335 5801
  5802  407 1470    0    0    0    0    0    0    0    0    0    0    0]
 [ 516 5803   13   49 1045  803  256    2    5   14 5804    5 1046   40
     4 1471 3297  458    4    1  113    0    0    0    0    0    0    0]
 [  31    1   31  242  195    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0]]


Several types of pretrained word embeddings exist, however we will be using the GloVe word embeddings from Stanford NLP since it is the most famous one and commonly used. The word embeddings can be obtained from Kaggle publict datasets.

In [25]:
embeddings_dictionary = dict()
glove_file = open('../input/glove6b/glove.6B.100d.txt', encoding="utf8")

In [26]:
# create word embedding vectors
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions

glove_file.close()

In [27]:
# Checking number of unique words
len(embeddings_dictionary.keys())

400000

In [28]:
# Constructing word embedding matrix only for required words
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [29]:
# Shape of our final embedding matrix
embedding_matrix.shape

(14721, 100)

In [30]:
# Building a simple sequential NN from finally constructed embedded matrix
model = Sequential()
embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 100)           1472100   
_________________________________________________________________
flatten_1 (Flatten)          (None, 2800)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2801      
Total params: 1,474,901
Trainable params: 2,801
Non-trainable params: 1,472,100
_________________________________________________________________
None


In [31]:
model.fit(padded_sentences, np.asarray(sentiments), epochs=100, verbose=1)


Epoch 1/100
139/139 [==============================] - 0s 1ms/step - loss: 0.2833 - acc: 0.9264
Epoch 2/100
139/139 [==============================] - 0s 1ms/step - loss: 0.2117 - acc: 0.9429
Epoch 3/100
139/139 [==============================] - 0s 1ms/step - loss: 0.1862 - acc: 0.9470
Epoch 4/100
139/139 [==============================] - 0s 1ms/step - loss: 0.1682 - acc: 0.9517
Epoch 5/100
139/139 [==============================] - 0s 1ms/step - loss: 0.1547 - acc: 0.9567
Epoch 6/100
139/139 [==============================] - 0s 1ms/step - loss: 0.1421 - acc: 0.9596
Epoch 7/100
139/139 [==============================] - 0s 1ms/step - loss: 0.1311 - acc: 0.9630
Epoch 8/100
139/139 [==============================] - 0s 1ms/step - loss: 0.1212 - acc: 0.9659
Epoch 9/100
139/139 [==============================] - 0s 1ms/step - loss: 0.1137 - acc: 0.9700
Epoch 10/100
139/139 [==============================] - 0s 1ms/step - loss: 0.1055 - acc: 0.9743
Epoch 11/100
139/139 [=================

In [32]:
loss, accuracy = model.evaluate(padded_sentences, np.asarray(sentiments), verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 99.977434


### Keras Functional Methods
In the last section, we saw how word embeddings can be used with the Keras sequential API. While the sequential API is a good starting point for beginners, as it allows you to quickly create deep learning models, it is extremely important to know how Keras Functional API works. Most of the advanced deep learning models involving multiple inputs and outputs use the Functional API.

In [33]:
deep_inputs = Input(shape=(length_long_sentence,))
embedding = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)(deep_inputs) # line A
flatten = Flatten()(embedding)
hidden = Dense(1, activation='sigmoid')(flatten)
model = Model(inputs=deep_inputs, outputs=hidden)

In [34]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 28, 100)           1472100   
_________________________________________________________________
flatten_2 (Flatten)          (None, 2800)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2801      
Total params: 1,474,901
Trainable params: 2,801
Non-trainable params: 1,472,100
_________________________________________________________________
None


In [35]:
model.fit(padded_sentences, np.asarray(sentiments), epochs=100, verbose=1)
loss, accuracy = model.evaluate(padded_sentences, np.asarray(sentiments), verbose=0)

print('Accuracy: %f' % (accuracy*100))

Epoch 1/100
139/139 [==============================] - 0s 1ms/step - loss: 0.2890 - acc: 0.9206
Epoch 2/100
139/139 [==============================] - 0s 1ms/step - loss: 0.2138 - acc: 0.9434
Epoch 3/100
139/139 [==============================] - 0s 1ms/step - loss: 0.1898 - acc: 0.9472
Epoch 4/100
139/139 [==============================] - 0s 1ms/step - loss: 0.1718 - acc: 0.9510
Epoch 5/100
139/139 [==============================] - 0s 1ms/step - loss: 0.1566 - acc: 0.9551
Epoch 6/100
139/139 [==============================] - 0s 1ms/step - loss: 0.1441 - acc: 0.9603
Epoch 7/100
139/139 [==============================] - 0s 1ms/step - loss: 0.1329 - acc: 0.9632
Epoch 8/100
139/139 [==============================] - 0s 1ms/step - loss: 0.1245 - acc: 0.9655
Epoch 9/100
139/139 [==============================] - 0s 1ms/step - loss: 0.1162 - acc: 0.9680
Epoch 10/100
139/139 [==============================] - 0s 1ms/step - loss: 0.1081 - acc: 0.9709
Epoch 11/100
139/139 [=================

Just like we were getting in Sequentional methods, here also we got 100% accuracy. As you can see, even though we can implement the word embedding on our own using Keras sequential and functional APIs, we prefer pre-trained word embeddings. Reason is pretty simple, we want to save our time by not re-inventing the wheel.

Here, we focused on word embedding implementations majorly, later we'll focus on solving Machine Translation and Text generation problems. Till then, happy learning (;

References: 
https://www.ijitee.org/wp-content/uploads/papers/v8i11/K13430981119.pdf

https://stackabuse.com/python-for-nlp-word-embeddings-for-deep-learning-in-keras/